In [3]:
from scipy import io
import os
import numpy as np
from scipy.sparse.linalg import eigsh
import random
import matplotlib.pyplot as plt
%matplotlib inline
import utils
clas = io.loadmat('./media/stimuli_class_assignment_confident.mat')
dat = io.loadmat('./media/natimg2800_M170714_MP032_2017-09-14.mat')

resp = dat['stim'][0]['resp'][0] # stim x neurons
spont = dat['stim'][0]['spont'][0] # timepts x neurons
# istim = (dat['stim'][0]['istim'][0]).astype(np.int32) # stim ids 
# istim -= 1 # get out of MATLAB convention
# istim = istim[:,0]
# nimg = istim.max() # these are blank stims (exclude them)
# resp = resp[istim<nimg, :]
# istim = istim[istim<nimg]

# # subtract spont (32D)
# mu = spont.mean(axis=0)
# sd = spont.std(axis=0) + 1e-6
# resp = (resp - mu) / sd
# spont = (spont - mu) / sd
# sv,u = eigsh(spont.T @ spont, k=32)
# resp = resp - (resp @ u) @ u.T

# # mean center each neuron
# resp -= resp.mean(axis=0)

In [3]:
print(resp.shape)
print(spont.shape)

(6980, 9476)
(335, 9476)


In [43]:
label=[]
for l in dat['stim'][0]['istim'][0]:
    label.append(l[0])
print(len(label))
print(len(set(label)))
print(label[0])

6980
2801
3


In [44]:
print(np.mean(l))
print(np.median(l))

2646.0
2646.0


In [45]:
len(clas["class_assignment"][0])

2800

- get images with the same label

In [46]:
imgs={}
for i in range(len(label)):
    try:    
        imgs[label [i]].append(i)
    except:
        imgs[label [i]] = [i]

#### Experiment to check the distance between neurons response for the same image. and printing the mean, std value, max, min distance

In [28]:
dists=[]
for key in imgs:
    if len(imgs[key])==2:
        dist=np.linalg.norm(resp[imgs[key][0]]-resp[imgs[key][1]])
    elif len(imgs[key])==3:
        dist1=np.linalg.norm(resp[imgs[key][0]]-resp[imgs[key][1]])
        dist2=np.linalg.norm(resp[imgs[key][0]]-resp[imgs[key][2]])
        dist3=np.linalg.norm(resp[imgs[key][1]]-resp[imgs[key][2]])
        dist=np.mean([dist1,dist2,dist3])
    dists.append(dist)

print(np.mean(dists))
print(np.std(dists))
print(max(dists))
print(min(dists))
        
        

12996.79
2300.0635
21409.63
8027.563


Check the same value for the same number of random neuron reponses 

In [29]:
m=len(resp)-1
dists=[]
for i in range(len(resp[0])):
    l1=random.randint(0,m)
    l2=random.randint(0,m)
    while l2 == l1:
        l2=random.randint(0,m)
        
    dist=np.linalg.norm(resp[l1]-resp[l2])
    dists.append(dist)

print(np.mean(dists))
print(np.std(dists))
print(max(dists))
print(min(dists))
        

14125.417
2673.4502
23472.658
6950.943


#### Experiments to relate neurons firing to each other. 

1. assuming that any value equals or above 1.0 means that a neuron fires. 

Check which neurons always fires when neuron "1" fires. 

In [5]:
import operator
n={}
for i in range (len(resp[0])):
    n[i]=0
    
le1=len(resp)
le2=len(resp[0])

for i in range(le1):
    
    tmp=[]
    for u in range(le2):
        if resp[i][1]>1.0:
            if resp[i][u]>1.0:
                n[u]+=1

sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
print(sorted_x[:5])

[(1, 4434), (8044, 3979), (5596, 3968), (3960, 3958), (122, 3936)]


Trying different 10 neurons 

In [10]:
le1=len(resp)
le2=len(resp[0])
for nu in range (2,12):
    n={}
    for i in range (len(resp[0])):
        n[i]=0

    for i in range(100):

        tmp=[]
        for u in range(le2):
            if resp[i][nu]>1.0:
                if resp[i][u]>1.0:
                    n[u]+=1

    sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
    print(sorted_x[:5])

[(2, 60), (8044, 59), (8125, 59), (5432, 58), (8969, 58)]
[(3, 58), (8044, 57), (8969, 57), (2969, 56), (7086, 56)]
[(4, 68), (2969, 67), (8044, 67), (281, 66), (385, 66)]
[(5, 58), (2969, 57), (8044, 57), (7086, 56), (7446, 56)]
[(6, 82), (7086, 81), (8044, 81), (8969, 81), (2969, 79)]
[(7, 42), (2969, 42), (8044, 42), (5432, 41), (6335, 41)]
[(8, 55), (5442, 54), (8044, 54), (2259, 53), (2969, 53)]
[(9, 70), (8044, 69), (7086, 68), (2969, 67), (6503, 67)]
[(10, 70), (8044, 69), (2969, 68), (7086, 68), (8791, 68)]
[(11, 60), (8044, 60), (8969, 60), (2969, 59), (8740, 59)]


Assuming that neuron fires only when its value is above 50 (the average)

In [67]:
n=set(range(len(resp[0])))

le1=len(resp)
le2=len(resp[0])

for i in range(le1):
    
    tmp=[]
    for u in range(le2):
        if resp[i][1]>50.0:
            if resp[i][u]>50.0:
                tmp.append(u)
    if resp[i][1]>50.0:
        n= n.intersection(tmp)
            

print(n)       

{1, 7805, 9222, 6327}


Assuming that neuron fires only when its value is above 5

In [78]:
import operator
n={}
for i in range (len(resp[0])):
    n[i]=0
    
le1=len(resp)
le2=len(resp[0])

for i in range(le1):
    
    tmp=[]
    for u in range(le2):
        if resp[i][1]>5.0:
            if resp[i][u]>5.0:
                n[u]+=1

sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
print(sorted_x[:5])

[(1, 2978), (8044, 2658), (3960, 2647), (122, 2640), (5596, 2633)]


Checking only if the difference between the values is less than 5.0 

In [83]:
import operator
n={}
for i in range (len(resp[0])):
    n[i]=0
    
le1=len(resp)
le2=len(resp[0])

for i in range(100): #100 images only
    
    tmp=[]
    for u in range(le2):
        if resp[i][1] >0.0 and resp[i][u]>0:
            if abs(resp[i][1] - resp[i][u])<=5.0:
                n[u]+=1

sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
print(sorted_x[:10])

[(1, 79), (664, 36), (7451, 36), (837, 35), (4340, 34)]


In [13]:
le1=len(resp)
le2=len(resp[0])
for nu in range (2,12):
    n={}
    for i in range (len(resp[0])):
        n[i]=0

    for i in range(100): #100 images only

        tmp=[]
        for u in range(le2):
            if resp[i][nu] >0.0 and resp[i][u]>0:
                if abs(resp[i][nu] - resp[i][u])<=5.0:
                    n[u]+=1

    sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
    print(sorted_x[:10])

[(2, 65), (4655, 21), (4986, 21), (7886, 21), (47, 20), (4987, 20), (5474, 20), (9332, 20), (3292, 19), (5677, 19)]
[(3, 62), (8641, 25), (664, 24), (4982, 24), (6793, 24), (8729, 24), (923, 23), (670, 22), (1179, 22), (1626, 22)]
[(4, 75), (4986, 36), (1416, 33), (4982, 32), (664, 31), (1328, 31), (574, 30), (670, 30), (5404, 30), (9101, 30)]
[(5, 66), (833, 29), (1244, 29), (2330, 27), (4423, 27), (4986, 27), (6529, 27), (7900, 27), (139, 26), (664, 26)]
[(6, 82), (3407, 19), (665, 18), (735, 17), (1911, 17), (5766, 17), (6078, 17), (6897, 17), (8483, 17), (416, 16)]
[(7, 53), (4986, 33), (833, 26), (4968, 25), (837, 24), (9333, 24), (928, 23), (2326, 23), (2870, 23), (5764, 23)]
[(8, 57), (928, 22), (1322, 22), (5404, 21), (3197, 20), (4648, 20), (7416, 20), (428, 19), (1634, 19), (7000, 19)]
[(9, 73), (9101, 22), (1397, 21), (1626, 21), (2634, 21), (7900, 21), (8148, 21), (166, 20), (1629, 20), (3224, 20)]
[(10, 73), (12, 20), (5495, 20), (8264, 20), (664, 18), (5193, 18), (14, 17)

### PCA

In [33]:
from sklearn.decomposition import PCA
X = np.array(resp)
pca = PCA(n_components=5000)
pca.fit(X)
pca.get_covariance()

array([[ 6.72874634e+02, -4.23329210e+00,  2.83707142e+01, ...,
         4.13456726e+02,  1.01351524e+02,  4.14291412e+02],
       [-4.23329306e+00,  6.70831421e+02,  5.21993446e+01, ...,
         1.41578827e+02,  3.72122116e+01,  1.32583328e+02],
       [ 2.83707180e+01,  5.21993484e+01,  1.27614453e+03, ...,
         2.90626587e+02,  1.23397301e+02,  8.03831238e+02],
       ...,
       [ 4.13456726e+02,  1.41578796e+02,  2.90626648e+02, ...,
         1.42641943e+04,  1.67661987e+03,  4.50692334e+03],
       [ 1.01351540e+02,  3.72122040e+01,  1.23397278e+02, ...,
         1.67662000e+03,  5.46494385e+03,  1.72566016e+03],
       [ 4.14291595e+02,  1.32583344e+02,  8.03831055e+02, ...,
         4.50692334e+03,  1.72566016e+03,  7.01242344e+04]], dtype=float32)

In [40]:
new_resp=pca.transform(resp)

In [41]:
new_resp.shape

(6980, 5000)

In [47]:
dists=[]
for key in imgs:
    if len(imgs[key])==2:
        dist=np.linalg.norm(new_resp[imgs[key][0]]-new_resp[imgs[key][1]])
    elif len(imgs[key])==3:
        dist1=np.linalg.norm(new_resp[imgs[key][0]]-new_resp[imgs[key][1]])
        dist2=np.linalg.norm(new_resp[imgs[key][0]]-new_resp[imgs[key][2]])
        dist3=np.linalg.norm(new_resp[imgs[key][1]]-new_resp[imgs[key][2]])
        dist=np.mean([dist1,dist2,dist3])
    dists.append(dist)

print(np.mean(dists))
print(np.std(dists))
print(max(dists))
print(min(dists))

12950.291
2316.559
21394.557
7909.606


In [48]:
m=len(new_resp)-1
dists=[]
for i in range(len(new_resp[0])):
    l1=random.randint(0,m)
    l2=random.randint(0,m)
    while l2 == l1:
        l2=random.randint(0,m)
        
    dist=np.linalg.norm(new_resp[l1]-new_resp[l2])
    dists.append(dist)

print(np.mean(dists))
print(np.std(dists))
print(max(dists))
print(min(dists))
        

14197.771
2707.0703
24412.375
6974.1885


In [49]:
le1=len(new_resp)
le2=len(new_resp[0])
for nu in range (2,12):
    n={}
    for i in range (len(new_resp[0])):
        n[i]=0

    for i in range(100): #100 images only

        tmp=[]
        for u in range(le2):
            if new_resp[i][nu] >0.0 and new_resp[i][u]>0:
                if abs(new_resp[i][nu] - new_resp[i][u])<=5.0:
                    n[u]+=1

    sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
    print(sorted_x[:10])

[(2, 88), (19, 2), (120, 2), (257, 2), (271, 2), (287, 2), (305, 2), (346, 2), (362, 2), (508, 2)]
[(3, 39), (160, 3), (580, 3), (607, 3), (792, 3), (895, 3), (976, 3), (1445, 3), (1634, 3), (1696, 3)]
[(4, 61), (1337, 4), (364, 3), (784, 3), (869, 3), (974, 3), (1290, 3), (1662, 3), (1859, 3), (2476, 3)]
[(5, 80), (14, 3), (410, 3), (1452, 3), (1512, 3), (1714, 3), (2900, 3), (12, 2), (33, 2), (47, 2)]
[(6, 89), (858, 3), (15, 2), (71, 2), (273, 2), (383, 2), (450, 2), (521, 2), (566, 2), (616, 2)]
[(7, 66), (44, 3), (64, 3), (192, 3), (209, 3), (281, 3), (545, 3), (1063, 3), (1761, 3), (2202, 3)]
[(8, 53), (673, 4), (132, 3), (162, 3), (454, 3), (471, 3), (519, 3), (1402, 3), (2031, 3), (2120, 3)]
[(9, 81), (53, 3), (197, 3), (1587, 3), (14, 2), (15, 2), (58, 2), (63, 2), (78, 2), (80, 2)]
[(10, 36), (630, 3), (664, 3), (949, 3), (1233, 3), (1284, 3), (2433, 3), (2434, 3), (2536, 3), (2694, 3)]
[(11, 45), (920, 4), (1038, 4), (1529, 4), (3425, 4), (3927, 4), (221, 3), (244, 3), (461,

In [52]:
import operator
n={}
for i in range (len(new_resp[0])):
    n[i]=0
    
le1=len(new_resp)
le2=len(new_resp[0])

for i in range(le1):
    
    tmp=[]
    for u in range(le2):
        if new_resp[i][1]>5.0:
            if new_resp[i][u]>5.0:
                n[u]+=1

sorted_x = list(sorted(n.items(), key=operator.itemgetter(1),reverse=True))
print(sorted_x[:5])

[(1, 3092), (12, 1602), (13, 1599), (94, 1594), (79, 1592)]
